In [1]:
#Select the first cell and press SHIFT+ENTER continuously to run each cell and go to the next
#You can queue inputs so press continuously till you reach the end

### Preprocessing

The provided code is designed to generate vector embeddings for images and texts extracted from a dataset. The resulting embeddings are then saved in the file "after_embeddings.csv," which is included in the provided zip file.

In the "Fashion_searcher.ipynb" notebook, you can expect to find code that allows you to input a query, whether it's an image or text, and use the pre-computed embeddings to compare it with the embeddings of the dataset.

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
from IPython.display import display
import requests
import glob
from io import BytesIO
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.chdir('./')
os.getcwd()

'/workspace'

In [4]:
from fashion_clip.fashion_clip import FashionCLIP
fclip = FashionCLIP('fashion-clip')

In [5]:
subset = pd.read_csv("/workspace/product_details-sheike-default-20230627052249.csv")

In [6]:
subset.head()

,__id,title,handle,description,skus,variant_skus,shopify_tags,images,media,options,...,sizegide,amount,_product_type,design,reklated,realted,relaated,rekated,relsted,styled_with
0,7028007567492,CASABLANCA DRESS,casablanca-dress-4,<p><span>Stand out at your next event in strik...,1005395-GRE-016\n1005395-GRE-014\n1005395-GRE-...,1005395-GRE-016\n1005395-GRE-014\n1005395-GRE-...,"bestseller-june23, bestseller-may23, care|1024...","[{""position"":1,""alt"":null,""width"":3000,""height...",[],"[{""id"":""gid://shopify/ProductOption/9037400473...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7064922652804,FANCIFUL DRESS,fanciful-dress-2,<p><span>Rich burgundy hues and silky satin fa...,1005561-PUR-018\n1005561-PUR-016\n1005561-PUR-...,1005561-PUR-018\n1005561-PUR-016\n1005561-PUR-...,"autumn-hues, autumnessential, care|10244, Colo...","[{""position"":1,""alt"":null,""width"":3000,""height...",[],"[{""id"":""gid://shopify/ProductOption/9082777206...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7016613019780,MEDUSA MAXI DRESS,medusa-maxi-dress-1,<p><span>Make a powerful entrance at your next...,1005344-BLA-016\n1005344-BLA-014\n1005344-BLA-...,1005344-BLA-016\n1005344-BLA-014\n1005344-BLA-...,"bestseller-june23, bold-edit, care| 10209, Col...","[{""position"":1,""alt"":null,""width"":3000,""height...",[],"[{""id"":""gid://shopify/ProductOption/9022465474...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7028007796868,ANGELICA MAXI DRESS,angelica-maxi-dress,<p><span>Make a lasting impression with the da...,1005401-BLU-018\n1005401-BLU-016\n1005401-BLU-...,1005401-BLU-018\n1005401-BLU-016\n1005401-BLU-...,"bold-edit, care| 10209, Colour-Blue, dress, dr...","[{""position"":1,""alt"":""model:16|ANGELICA MAXI D...",[],"[{""id"":""gid://shopify/ProductOption/9037400899...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7035457405060,MILLENIUM DRESS,millenium-dress-5,<p><span>Corset detailing meets effortless sty...,1005458-GRY-016\n1005458-GRY-014\n1005458-GRY-...,1005458-GRY-016\n1005458-GRY-014\n1005458-GRY-...,"autumn-hues, autumnessential, basic, care| 102...","[{""position"":1,""alt"":null,""width"":3000,""height...",[],"[{""id"":""gid://shopify/ProductOption/9047198498...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = subset.loc[subset['available']]
df.dropna(subset=['image_url'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(len(df))

730


/opt/conda/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
df['p.m.filter.colour'] = df['p.m.filter.colour'].str.replace('\n', ' ')
df['p.m.filter.style'] = df['p.m.filter.style'].str.replace('\n', ' ')
df['p.m.filter.features'] = df['p.m.filter.features'].str.replace('\n', ' ')
df['p.m.filter.category'] = df['p.m.filter.category'].str.replace('\n', ' ')
df['p.m.filter.occasion'] = df['p.m.filter.occasion'].str.replace('\n', ' ')

/tmp/ipykernel_138/2358446456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['p.m.filter.colour'] = df['p.m.filter.colour'].str.replace('\n', ' ')
/tmp/ipykernel_138/2358446456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['p.m.filter.style'] = df['p.m.filter.style'].str.replace('\n', ' ')
/tmp/ipykernel_138/2358446456.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [9]:
df['text'] = ''

for index, row in df.iterrows():
    # Concatenate the strings from the four columns with a space separator
    merged_text = ' '.join([str(row['title']+ str(":")), str(row['p.m.filter.colour']), str(row['p.m.filter.style']),
                            str(row['p.m.filter.features']), str(row['p.m.filter.occasion']), str(row['p.m.filter.category'])])

    merged_text = merged_text.replace('nan', '')
    
    df.at[index, 'text'] = merged_text

/tmp/ipykernel_138/3552887762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = ''


In [10]:
df['text'].head()

0    CASABLANCA DRESS: Green  A Line Maxi Sleeveles...
1    FANCIFUL DRESS: Red  Wrap style Long Sleeve Mi...
2    MEDUSA MAXI DRESS: Black  Maxi Slim Long Sleev...
3    ANGELICA MAXI DRESS: Blue  Long Sleeve Slim V ...
4    MILLENIUM DRESS: Grey  Long Sleeve Midi Slim  ...
Name: text, dtype: object

In [12]:
import requests

def check_link_validity(links):
    invalid_links = []

    for link in links:
        try:
            response = requests.head(link)
            if response.status_code != 200:
                invalid_links.append(link)
        except requests.exceptions.RequestException:
            invalid_links.append(link)

    return invalid_links

# Example list of links (replace this with your own 730 links)
links_to_check = df['image_url'].tolist()

invalid_links = check_link_validity(links_to_check)

print("\nInvalid Links:")
for link in invalid_links:
    print(link)



Invalid Links:
https://cdn.shopify.com/s/files/1/0285/9001/4596/files/1005189_AMALFI_LONGLINE_KNIT_TOP_BLACK_LISA_2_400x400.jpg?v=1687754348
https://cdn.shopify.com/s/files/1/0285/9001/4596/products/S54-1005149_DREAMER_KNOT_TOP_BLACK-23088-Sheike-D2-1263_400x400.jpg?v=1682924586
https://cdn.shopify.com/s/files/1/0285/9001/4596/products/S45-1005185_WOOL_BLEND_BASIC_TOP_OATMEAL-23088-Sheike-D2-1626_400x400.jpg?v=1685503657
https://cdn.shopify.com/s/files/1/0285/9001/4596/products/1004353_EMILY_SKIRT_BLUSH_NINA_4_400x400.jpg?v=1682924514
https://cdn.shopify.com/s/files/1/0285/9001/4596/files/Sheike_EGiftCard__BestWishes_1500x2000px_400x400.jpg?v=1685309725


In [13]:
df = df[~df['image_url'].isin(invalid_links)]

In [38]:
#THIS FILE WILL TAKE ~2MINS TO RUN DEPENDING ON YOUR NET SPEED
from tqdm import tqdm

images = []
total_urls = len(df['image_url'])

# Use tqdm to create a progress bar
with tqdm(total=total_urls, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}') as pbar:
    for url in df['image_url'].tolist():
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        images.append(image)

        # Update the progress bar
        pbar.update(1)

# Print completion message
print("Image download completed!")

100%|██████████████████████████████████████████████████████████████████| 725/725

Image download completed!


In [39]:
#THIS CELL WILL TAKE ~45MINS TO RUN ON CPU
image_embeddings = []
with tqdm(total=725, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}') as pbar:
    for i in range(725):
        embedding = fclip.encode_images([images[i]], batch_size=1)[0]
        embedding /= np.linalg.norm(embedding, ord=2)
        image_embeddings.append(embedding.tolist())
        pbar.update(1)
# Print completion message
print("Image embeddings calculation completed!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 78.79it/s]
100%|██████████████████████████████████████████████████████████████████| 725/725

Image embeddings calculation completed!


In [16]:
#THIS FILE WILL TAKE ~2MINS TO RUN DEPENDING ON YOUR NET SPEED
from tqdm import tqdm
import json

last_images_url = []
last_images = []

for i in df['images'].tolist()[:2]:
    data = json.loads(i)
    last_url = data[-1]["src"]
    last_images_url.append(last_url)
    
# Use tqdm to create a progress bar
with tqdm(total=total_urls, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}') as pbar:
    for url in last_images_url:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        last_images.append(image)

        # Update the progress bar
        pbar.update(1)

# Print completion message
print("Image download completed!")

725
['https://cdn.shopify.com/s/files/1/0285/9001/4596/files/S51-1005395_CASABLANCA_MAXI_DRESS_GREEN-23373-Sheike-B-0260.jpg?v=1687754384', 'https://cdn.shopify.com/s/files/1/0285/9001/4596/files/S46-1005561_FANCIFUL_DRESS_WINE-23373-Sheike-B-0231.jpg?v=1687754415']


In [ ]:
#THIS CELL WILL TAKE ~45MINS TO RUN ON CPU
last_image_embeddings = []
with tqdm(total=725, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}') as pbar:
    for i in range(725):
        embedding = fclip.encode_images([last_images[i]], batch_size=1)[0]
        embedding /= np.linalg.norm(embedding, ord=2)
        last_image_embeddings.append(embedding.tolist())
        pbar.update(1)
# Print completion message
print("Image embeddings calculation completed!")

In [40]:
df['image_embeddings'] = image_embeddings
df['last_image_embeddings'] = last_image_embeddings

In [21]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large')
model = AutoModel.from_pretrained('intfloat/e5-large')

In [22]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def embed_query(data):
  batch_dict = tokenizer(data, max_length=512, padding=True, truncation=True, return_tensors='pt')
  output = model(**batch_dict)
  embeddings = average_pool(output.last_hidden_state, batch_dict['attention_mask'])
  embeddings = F.normalize(embeddings, p=2, dim=1)
  embeddings = embeddings.tolist()
  return embeddings

In [23]:
columns_to_process = ['title','p.m.filter.colour','p.m.filter.style','p.m.filter.features','p.m.filter.occasion','p.m.filter.category']
new_columns = ['text_embeddings_title','text_embeddings_color','text_embeddings_style','text_embeddings_features','text_embeddings_occasion','text_embeddings_category']
batch_size=100

for column, new_column in zip(columns_to_process, new_columns):
    column_texts = df[column].astype(str).tolist()
    embed = [embed_query(i)[0] for i in column_texts]
    df[new_column] = embed
    print(column)

title
p.m.filter.colour
p.m.filter.style
p.m.filter.features
p.m.filter.occasion
p.m.filter.category


In [31]:
df.drop(['handle', 'description', 'skus', 'variant_skus',
       'shopify_tags','media', 'options', 'variants',
       'compare_at_price', 'price', 'sale_price_max', 'link', 'in_stock',
       'available', 'published_at', 'updated_at', 'created_at', 'synced_at',
       '__inventory_total', '__inventory_average', '__inventory_min',
       '__inventory_max', 'product_status', '_vendor',
       '_collections', 'colour', 'size', 'promo', '_tags', '__size_all',
        'available_on_online_store', 'p.m.okendo.summaryData.reviewCount',
       'p.m.okendo.summaryData.reviewAverageValue', 'p.m.badge.custom',
       'related', 'sizeguide', 'sizegide', 'amount', '_product_type', 'design',
       'reklated', 'realted', 'relaated', 'rekated', 'relsted', 'styled_with'],axis=1,inplace=True)

In [36]:
type(df['text_embeddings_title'][0])

list

In [41]:
#This CSV contains image and text ebeddings
df.to_csv('test_embeddings.csv', index=False)